## 1. 환경 설정

In [1]:
# (1) Env 환경변수
from dotenv import load_dotenv
load_dotenv()

# (2) 기본 라이브러리
import os
from glob import glob

from pprint import pprint

# 벡터저장소 로드
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
)

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


벡터 저장소에 저장된 문서 수: 5


## 2. LangChain LCEL
### 2.1 Prompt + LLM

In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4.1-nano", 
    temperature=0.3, 
    max_tokens=100,
    )

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

# 템플릿 입력 변수를 출력
print(prompt.input_variables)

# input 값을 전달하여 프롬프트를 랜더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")
print(prompt_text)

# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)
print(response.content)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]
['query']
System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?
테슬라의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다. 이후 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 그리고 이반스 리우(Ivan Sroulel)가 합류하면서 회사의 핵심 멤버가 되었습니다. 특히 일론 머스크는 테슬라의 대표이사로서 회사의 성장과 발전에


In [5]:
# LCEL 체인을 구성
chain = prompt | llm
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1449531d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x144950f50>, root_client=<openai.OpenAI object at 0x144917c90>, root_async_client=<openai.AsyncOpenAI object at 0x120798e50>, model_name='gpt-4.1-nano', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [6]:
# 체인의 입력 스키마를 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [7]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})
print(response.content)

테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타페니스(Marc Tarpenning), 그리고 이반스 리우(Ivan Sutherland)입니다. 이 중에서 일론 머스크는 종종 테슬라의 대표적 인물로 알려져 있으며, 현재도 회사의 중요한 역할을 맡고 있습니다.


In [22]:
# 체인을 실행 - 옵션 2
response = chain.invoke("테슬라 창업자는 누구인가요?")
print(response.content)

테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타펜링(Marc Tarpenning), 그리고 이반스 리브체크(Ivaniviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviviv


## 2.2 Prompt + LLM + Output Parser
### a) 문자열 파싱 - StrOutputParser

In [ ]:

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

str_chain = prompt | llm | output_parser
query="리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)
print(str_response)

```json
{
  "name": "리비안",
  "title": "설립년도"
}
```


### b) JSON 출력 - JsonOutputParser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행(Json 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON형식으로 출력해주세요.")
print(json_response)

json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": "마틴 에반스, 마크 타페닝, JB 스트라우벨, 이반스, 마틴 에반스, 마크 타페닝, JB 스트라우벨"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 34, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'finish_reason': 'stop', 'logprobs': None} id='run-e72cedd0-29f0-47d5-9bf5-dad29e83c099-0' usage_metadata={'input_tokens': 34, 'output_tokens': 51, 'total_tokens': 85}
{'창업자': '마틴 에반스, 마크 타페닝, JB 스트라우벨, 이반스, 마틴 에반스, 마크 타페닝, JB 스트라우벨'}


### c) Schema 지정 - Pydantic

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")

# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person)
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions())
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")

# 체인을 구성
person_chain = prompt | llm | person_parser
response = person_chain.invoke("테슬라 창업자는 누구인가요?")
print(response)



PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title or position of the person.", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema

## 3. Chat Completion Methods
### (1) stream
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [18]:
import time

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # print 함수는 출력할 때마다 줄바꿈을 하지만, 줄바꿈 없이 하려면 end=""를 사용
    # flush=Ture 옵션을 사용하여 출력 버퍼를 즉시 비움. 데이터를 지연 없이 즉시 출력하는데 유용
    print(chunk.content, end="", flush=True)
    time.sleep(0.1) # 0.1초 대기(100ms)

테슬라의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다. 이후 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 그리고 이반스 리브스(Ivaniv Reeve)가 참여하여 회사의 발전에 중요한 역할을 했습니다. 특히 일론 머스크는 테슬라의 CEO로서 회사의 성장과 전기차 산업의

### (2) batch
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [24]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?"
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창업자는 일론 머스크(Elon Musk), JB 스트라우벨(JB Straubel), 마틴 에버하드(Martin Eberhard), 마크 타페니스(Marc Tarpenning), 그리고 이반스 리브(Ivan Sutherland)입니다. 이 중에서 일론 머스크는 종종 대표 창업자로 언급되며, 회사의 성장과 방향성에 큰 영향을 미쳤습니다.

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ Scaringe입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 및 전기 픽업트럭 개발에 주력하고 있습니다.

